In [1]:
## download songlyrics.zip from Kaggle, requires log in
## https://www.kaggle.com/mousehead/songlyrics/downloads/songlyrics.zip

In [2]:
import pandas as pd

from gensim.models import Word2Vec
from nltk import sent_tokenize, word_tokenize

In [3]:
data = pd.read_csv('./data/kaggle_lyrics/songlyrics.zip')
data.drop(columns=['link'], inplace=True)
data.head()

,artist,song,text
0,ABBA,Ahe's My Kind Of Girl,"Look at her face, it's a wonderful face \nAnd..."
1,ABBA,"Andante, Andante","Take it easy with me, please \nTouch me gentl..."
2,ABBA,As Good As New,I'll never know why I had to go \nWhy I had t...
3,ABBA,Bang,Making somebody happy is a question of give an...
4,ABBA,Bang-A-Boomerang,Making somebody happy is a question of give an...


In [4]:
## use kaggle data set on 57,650 songs (check if it has artists for text generation)

artist_names = ['Eric Clapton', 'Jimi Hendrix', 'Bob Dylan', 'Muddy Waters', 'Eagles']
for artist_name in artist_names:
    print('There are {:03d} songs set for {}.'.format(
        len(data[data['artist'].str.contains(artist_name)]), artist_name))

There are 152 songs set for Eric Clapton.
There are 127 songs set for Jimi Hendrix.
There are 188 songs set for Bob Dylan.
There are 000 songs set for Muddy Waters.
There are 041 songs set for Eagles.


In [5]:
data['text'][0]

"Look at her face, it's a wonderful face  \nAnd it means something special to me  \nLook at the way that she smiles when she sees me  \nHow lucky can one fellow be?  \n  \nShe's just my kind of girl, she makes me feel fine  \nWho could ever believe that she could be mine?  \nShe's just my kind of girl, without her I'm blue  \nAnd if she ever leaves me what could I do, what could I do?  \n  \nAnd when we go for a walk in the park  \nAnd she holds me and squeezes my hand  \nWe'll go on walking for hours and talking  \nAbout all the things that we plan  \n  \nShe's just my kind of girl, she makes me feel fine  \nWho could ever believe that she could be mine?  \nShe's just my kind of girl, without her I'm blue  \nAnd if she ever leaves me what could I do, what could I do?\n\n"

In [6]:
%%time
data['text'] = data['text'].apply(lambda text: word_tokenize(text.replace('\n', '. ')))

CPU times: user 4min 14s, sys: 1.65 s, total: 4min 15s
Wall time: 4min 16s


In [7]:
data['text'][0]

['Look',
 'at',
 'her',
 'face',
 ',',
 'it',
 "'s",
 'a',
 'wonderful',
 'face',
 '.',
 'And',
 'it',
 'means',
 'something',
 'special',
 'to',
 'me',
 '.',
 'Look',
 'at',
 'the',
 'way',
 'that',
 'she',
 'smiles',
 'when',
 'she',
 'sees',
 'me',
 '.',
 'How',
 'lucky',
 'can',
 'one',
 'fellow',
 'be',
 '?',
 '.',
 '.',
 'She',
 "'s",
 'just',
 'my',
 'kind',
 'of',
 'girl',
 ',',
 'she',
 'makes',
 'me',
 'feel',
 'fine',
 '.',
 'Who',
 'could',
 'ever',
 'believe',
 'that',
 'she',
 'could',
 'be',
 'mine',
 '?',
 '.',
 'She',
 "'s",
 'just',
 'my',
 'kind',
 'of',
 'girl',
 ',',
 'without',
 'her',
 'I',
 "'m",
 'blue',
 '.',
 'And',
 'if',
 'she',
 'ever',
 'leaves',
 'me',
 'what',
 'could',
 'I',
 'do',
 ',',
 'what',
 'could',
 'I',
 'do',
 '?',
 '.',
 '.',
 'And',
 'when',
 'we',
 'go',
 'for',
 'a',
 'walk',
 'in',
 'the',
 'park',
 '.',
 'And',
 'she',
 'holds',
 'me',
 'and',
 'squeezes',
 'my',
 'hand',
 '.',
 'We',
 "'ll",
 'go',
 'on',
 'walking',
 'for',
 'hours',
